In [2]:
from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [4]:
number_of_tweets = 500
tweets = []
user_id =[]
created_at = []
keyword_list = ['News', 'Stock'] # , 'Customers', 'Employees', 'Company', 'Price', 'Shareholders', 'Market', 'Soaring', 'Trends'
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

Amazon News
Amazon Stock


In [5]:
df = pd.DataFrame({"Created_at": created_at,"User_id": user_id,"Tweets":tweets})
df

,Created_at,User_id,Tweets
0,2022-02-07 08:44:37,210955835,RT @Fridays4future: “The oil spill has reached...
1,2022-02-07 08:44:18,4419993470,Stock Market Today: Amazon's Historic Day Push...
2,2022-02-07 08:43:49,1299877337148936192,WOW: Priyanka Chopra Jonas to be seen in Holly...
3,2022-02-07 08:42:50,15482079,Net sales at Amazon were up by 9% year on year...
4,2022-02-07 08:42:50,1267197215635726336,&lt;Ethereum Price&gt;\nETH/USDT(Binance): 306...
...,...,...,...
995,2022-02-07 01:23:14,1377519267143376898,PowerColor Red Dragon AMD Radeon™ RX 6800 Gami...
996,2022-02-07 01:22:43,499672969,(US)In Stock: GIGABYTE AORUS 3070 Ti Master - ...
997,2022-02-07 01:21:46,1377519267143376898,Xbox Series S is now in stock. \nProduct Page:...
998,2022-02-07 01:20:59,1418587410384908293,(US) In Stock: GIGABYTE AORUS GeForce RTX 3080...


In [6]:
df = df[~df.Tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [7]:
def clean_up_tweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text) # Remove mentions @
    text = re.sub(r'#', '', text) # Remove hastag symbol
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text) # Remove hyper link
    text = re.sub('[^A-Za-z0-9]+', ' ', text) # Remove special characters
    text = re.sub(r'[0-9]+', '', text)
    #Remove stop words
    res = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            res += WordNetLemmatizer().lemmatize(word) + ' '

    return res

In [8]:
df['Tweets'] = df['Tweets'].apply(clean_up_tweet)
df = df.drop_duplicates(subset='Tweets')
df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets
0,2022-02-07 08:44:18,4419993470,Stock Market Today Amazon Historic Day Pushes ...
1,2022-02-07 08:43:49,1299877337148936192,WOW Priyanka Chopra Jonas seen Hollywood actio...
2,2022-02-07 08:42:50,15482079,Net sale Amazon year year bn bn three month De...
3,2022-02-07 08:42:50,1267197215635726336,lt Ethereum Price gt ETH USDT Binance Check Pr...
4,2022-02-07 08:41:32,1483083938151428096,Do like Amazon Gift Card Great news This Amazo...
...,...,...,...
317,2022-02-07 01:35:45,1306463816226660352,Amazon ASUS ROG Strix Z A Gaming WiFi D LGA AT...
318,2022-02-07 01:30:44,780460754910732288,The Art Soul Dune back stock sale Order
319,2022-02-07 01:27:53,52126417,Amazon Stock Saves Stock Market Stock Market A...
320,2022-02-07 01:23:24,1309851202511831041,ad MSI Radeon RX XT Gaming X Trio Carte Graph ...


In [13]:
# Create function to get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create function to get polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Function compute negative, positive
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets,Subjectivity,Polarity
0,2022-02-07 08:44:18,4419993470,Stock Market Today Amazon Historic Day Pushes ...,0.250000,0.125000
1,2022-02-07 08:43:49,1299877337148936192,WOW Priyanka Chopra Jonas seen Hollywood actio...,0.366667,0.066667
2,2022-02-07 08:41:32,1483083938151428096,Do like Amazon Gift Card Great news This Amazo...,0.583333,0.566667
3,2022-02-07 08:37:31,1352234446402048003,Businesshala News Exclusive Amazon Other Poten...,0.687500,-0.062500
4,2022-02-07 08:36:34,1156926992119488513,S amp P rise end best week year far Amazon pus...,0.600000,0.450000
...,...,...,...,...,...
170,2022-02-07 02:31:04,1006334607946649600,Top Stock Market Moving Headlines From Last We...,0.355556,0.333333
171,2022-02-07 02:22:43,14562685,MyPOV fire sale What install base worth darlin...,0.700000,-0.166667
172,2022-02-07 02:14:20,2761384656,Do know people put investment capital Googles ...,0.650000,0.350000
173,2022-02-07 01:52:43,20110439,amazon stock shift frequently right listing go...,0.433929,0.371429


In [14]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(2,3))
# matrix of n_grams
ngrams = c_vec.fit_transform(df['Tweets'])
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)
# list of ngrams
vocab = c_vec.vocabulary_
df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'Frequency', 1:'n-gram'})

df_ngram = df_ngram.drop_duplicates(subset='n-gram')
df_ngram = df_ngram.reset_index(drop=True)
df_ngram['Polarity'] = df_ngram['n-gram'].apply(lambda x: TextBlob(x).polarity)
df_ngram['Subjective'] = df_ngram['n-gram'].apply(lambda x: TextBlob(x).subjectivity)
df_ngram.drop(df_ngram.loc[df_ngram['Polarity'] == 0.0].index, inplace=True)
df_ngram = df_ngram.reset_index(drop=True)
df_ngram

,Frequency,n-gram,Polarity,Subjective
0,6,top company exaggerating,0.5,0.500
1,6,top company,0.5,0.500
2,5,good news,0.7,0.600
3,5,climate change top,0.5,0.500
4,5,change top company,0.5,0.500
...,...,...,...,...
1089,1,account kindly give,0.6,0.900
1090,1,account kindly,0.6,0.900
1091,1,able spend buy,0.5,0.625
1092,1,able spend,0.5,0.625


In [47]:
from sklearn.feature_extraction.text import CountVectorizer
cv_vect = CountVectorizer(ngram_range=(2,3))
cv = cv_vect.fit_transform(df['Tweets'])


In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

y = df['Polarity'].values.astype('float')

X_train, X_test, y_train, y_test = train_test_split(cv, y, random_state=42, test_size=0.2)
y_test


array([ 0.5       ,  0.375     ,  0.18333333,  0.2       ,  0.15      ,
        0.2       , -0.125     , -0.25      ,  0.16666667,  0.46666667,
        0.8       ,  0.15      ,  0.06818182,  1.        ,  0.06666667,
        0.2       ,  0.23928571,  0.65      ,  0.2       , -0.26666667,
        0.1       , -0.25      ,  0.56666667,  0.11818182,  0.65      ,
        0.5       ,  0.8       ,  0.25      ,  0.27878788, -0.175     ,
        0.125     ,  0.075     ,  0.1       ,  0.5       ,  0.13636364])

In [71]:
from sklearn.metrics import classification_report
import numpy as np
from sklearn import linear_model
from sklearn import svm

classifiers = [
    svm.SVR(),
    linear_model.SGDRegressor(),
    linear_model.LinearRegression()]

trainingData    = X_train
trainingScores  = y_train
predictionData  = X_test

for item in classifiers:
    print(item)
    clf = item
    clf.fit(trainingData, trainingScores)
    print(y_test)
    print(clf.predict(predictionData),'\n')

SVR()
[ 0.5         0.375       0.18333333  0.2         0.15        0.2
 -0.125      -0.25        0.16666667  0.46666667  0.8         0.15
  0.06818182  1.          0.06666667  0.2         0.23928571  0.65
  0.2        -0.26666667  0.1        -0.25        0.56666667  0.11818182
  0.65        0.5         0.8         0.25        0.27878788 -0.175
  0.125       0.075       0.1         0.5         0.13636364]
[0.16220426 0.16261809 0.16269859 0.15999916 0.1624147  0.16237755
 0.16244308 0.15775864 0.14608475 0.16282895 0.32423854 0.19167314
 0.16354009 0.29105801 0.16303811 0.16249208 0.16259408 0.23675246
 0.16320363 0.16264345 0.16308071 0.19003952 0.1624309  0.16290576
 0.19645291 0.18122498 0.16418044 0.16286629 0.16905635 0.16893559
 0.16331382 0.16261809 0.20911678 0.1633787  0.20734425] 

SGDRegressor()
[ 0.5         0.375       0.18333333  0.2         0.15        0.2
 -0.125      -0.25        0.16666667  0.46666667  0.8         0.15
  0.06818182  1.          0.06666667  0.2        